In [27]:
import os,sys
import pandas as pd

In [28]:
sample_info = pd.DataFrame(columns=["sample_name", "trimed_forward_fq", "trimed_reverse_fq"])

data_path = os.path.abspath("../data") + "/"
trimed_path = os.path.abspath("../trimmed/") + "/"
for path in os.listdir(data_path):
    # print(os.path.abspath(path))
    if os.path.isdir(data_path + path):
        sample_name = path
        trimed_forward_fq = trimed_path + sample_name +"_trimed_R1.fastq.gz"
        # print(trimed_forward_fq)
        trimed_reverse_fq = trimed_path + sample_name +"_trimed_R2.fastq.gz"
        sample_info = pd.concat([sample_info, pd.DataFrame([[sample_name, trimed_forward_fq, trimed_reverse_fq]], columns=["sample_name", "trimed_forward_fq", "trimed_reverse_fq"])])

sample_info = sample_info.reset_index(drop=True)

In [29]:
# 去掉 公共数据
public_sample_info = sample_info.loc[ sample_info["sample_name"].str.contains("SRR"), :]
sample_info = sample_info.loc[~sample_info["sample_name"].str.contains("SRR"), :]

In [6]:
sample_info.loc[sample_info["sample_name"].str.contains("RP") | sample_info["sample_name"].str.contains("siNC-1-T") , :]

,sample_name,trimed_forward_fq,trimed_reverse_fq
1,smmc-siYB1-2-RP,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...
2,smmc-siNC-1-T,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...


In [21]:
public_sample_info

,sample_name,trimed_forward_fq,trimed_reverse_fq
1,SRR1630831,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...


## Bowtie2 比对

In [8]:
refernce_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/bowtie2_index/GCA_000001405.29_GRCh38.p14_genomic"

# 根据要求之分析两个样本
sample_info_select = sample_info.loc[sample_info["sample_name"].str.contains("RP") | sample_info["sample_name"].str.contains("siNC-1-T") , :]
# sample_info_select.iloc[0,] = [sample_info_select.iloc[0,]["sample_name"], "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R1_001.fastq.gz", "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R2_001.fastq.gz"]

def rum_sbatch(x):
    # 线程数在这里修改
    command = f'sbatch step2_mapping.sh -i {x["trimed_forward_fq"]} -I {x["trimed_reverse_fq"]} -t 12 -n {x["sample_name"]} -r {refernce_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info_select.apply(rum_sbatch, axis=1)

process sample :  smmc-siYB1-2-RP
Submitted batch job 26508737
process sample :  smmc-siNC-1-T
Submitted batch job 26508738


1    None
2    None
dtype: object

### 换成 genecode 的 index 再试一下

In [11]:
sample_info["sample_name"] = sample_info["sample_name"] + "_genecode"

In [12]:
refernce_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/bowtie2_index/GRCh38.p13.genome"


# 只分析 ribo-seq 样本
sample_info_select = sample_info.iloc[[1],:] 
# sample_info_select.iloc[0,] = [sample_info_select.iloc[0,]["sample_name"], "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R1_001.fastq.gz", "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R2_001.fastq.gz"]

def rum_sbatch(x):
    # 线程数在这里修改
    command = f'sbatch step2_mapping.sh -i {x["trimed_forward_fq"]} -I {x["trimed_reverse_fq"]} -t 12 -n {x["sample_name"]} -r {refernce_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info_select.apply(rum_sbatch, axis=1)

process sample :  smmc-siYB1-2-RP_genecode
Submitted batch job 26517011


1    None
dtype: object

## STAR 比对

* 目前来看，后续想用 RiboCode call ORF ，必须用 STAR 进行比对

In [17]:
reference_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/STAR_index/"

sample_info_star = sample_info.copy()
sample_info_star["sample_name"] = sample_info_star["sample_name"] + "_star"
# sample_info_star.iloc[0,1] = sample_info_star.iloc[0,1].replace(".gz", "")

def rum_sbatch(x):
    # 线程数在这里修改
    thread = 16
    command = f'sbatch step2_mapping_star.sh -i {x["trimed_forward_fq"]} -I  {x["trimed_reverse_fq"]}  -t {thread} -n {x["sample_name"]} -r {reference_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info_star.apply(rum_sbatch, axis=1)

process sample :  smmc-siYB1-2-RP_star
Submitted batch job 26528700


0    None
dtype: object

In [30]:
# 单端测序，然后再加上样品名
public_sample_info = public_sample_info.drop(columns=["trimed_reverse_fq"]) # 单端测序
public_sample_info["sample_name"] = public_sample_info["sample_name"] + "_star"

In [32]:
reference_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/STAR_index/"


# sample_info_star.iloc[0,1] = sample_info_star.iloc[0,1].replace(".gz", "")

# 这个示例数据是单端测序
def rum_sbatch(x):
    # 线程数在这里修改
    thread = 16
    command = f'sbatch step2_mapping_star.sh -i {x["trimed_forward_fq"]}   -t {thread} -n {x["sample_name"]} -r {reference_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

public_sample_info.apply(rum_sbatch, axis=1)

process sample :  SRR1630831_star
Submitted batch job 26529272


1    None
dtype: object

In [31]:
public_sample_info["sample_name"] = public_sample_info["sample_name"] + "_star"

public_sample_info

,sample_name,trimed_forward_fq
1,SRR1630831_star,/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/...
